In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/skin_detection_1271')

Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
image_info = pd.read_csv('processed_data/ISIC_2019_Training_GroundTruth_Processed_Balanced.csv')

In [5]:
image_info

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,Cat,y
0,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
1,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
2,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
3,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
4,ISIC_0000007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24801,ISIC_0073153_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24802,ISIC_0073157_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24803,ISIC_0073198_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24804,ISIC_0073214_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1


In [6]:
def get_image(df):
    return np.asarray(Image.open((f"processed_data/{df['Cat']}/{df['image']}.jpg")), dtype=np.float32)

In [7]:
data_size =1000

In [8]:
X = np.array(image_info.sample.apply(get_image, axis=1))

In [9]:
X = np.stack(X)

In [10]:
y = image_info['y'][:data_size]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Transfered model -VGG

In [12]:
def load_model():
    model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
    return model

In [13]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    model.trainable = False

    return model

In [14]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''

    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(200, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [15]:
def build_model():

    model = load_model()
    model = add_last_layers(model)

    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy', 'Recall'])
    return model

In [16]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 12, 12, 512)       14714688  
                                                                 
 flatten (Flatten)           (None, 73728)             0         
                                                                 
 dense (Dense)               (None, 200)               14745800  
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 29,460,689
Trainable params: 14,746,001
Non-trainable params: 14,714,688
_________________________________________________________________


In [17]:
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [18]:
model = build_model()

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)


In [19]:
history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    epochs=10,
                    batch_size=32,
                    callbacks=[es])

Epoch 1/10
16/16 [==============================] - 56s 2s/step - loss: 8.7900 - accuracy: 0.6053 - recall: 0.3623 - val_loss: 2.0809 - val_accuracy: 0.6540 - val_recall: 0.0135
Epoch 2/10
16/16 [==============================] - 11s 682ms/step - loss: 0.6545 - accuracy: 0.8016 - recall: 0.5000 - val_loss: 0.6428 - val_accuracy: 0.5592 - val_recall: 0.8919
Epoch 3/10
16/16 [==============================] - 13s 810ms/step - loss: 0.3640 - accuracy: 0.7771 - recall: 0.7899 - val_loss: 0.7421 - val_accuracy: 0.7156 - val_recall: 0.2432
Epoch 4/10
16/16 [==============================] - 12s 790ms/step - loss: 0.1799 - accuracy: 0.9080 - recall: 0.7101 - val_loss: 0.8208 - val_accuracy: 0.6967 - val_recall: 0.4730
Epoch 5/10
16/16 [==============================] - 12s 786ms/step - loss: 0.0934 - accuracy: 0.9448 - recall: 0.8188 - val_loss: 0.8495 - val_accuracy: 0.7156 - val_recall: 0.3108
Epoch 6/10
16/16 [==============================] - 12s 788ms/step - loss: 0.0581 - accuracy: 0.97

### Evaluate on test set ###

In [20]:
print(model.evaluate(X_test, y_test, verbose=1))

10/10 [==============================] - 9s 970ms/step - loss: 0.8629 - accuracy: 0.7867 - recall: 0.5165
[0.8628613352775574, 0.7866666913032532, 0.5164835453033447]
